In [10]:
import json
import os
import time
import ipyparallel as ipp

import matplotlib.pyplot as plt
import torch
import dfca as DFCA
import ifca as IFCA

import numpy as np

from util import *

Reads Config file and prepares the arguments you can choose in the config_dfca.json/config_ifca.json

In [11]:
rc = ipp.Cluster(n=2).start_and_connect_sync()
rc.activate()

rc.ids 

dview = rc[:]

Starting 2 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/2 [00:00<?, ?engine/s]

In [12]:
seeds = [11,12,13,14,15]

dfca_li = []
dfca_gi = []
ifca = []

In [13]:
%%px --targets all

import sys
import os
sys.path.append(os.getcwd())

seeds = [11,12,13,14,15]

dfca_gi = []
ifca = []

Running the dfca-gi experiment

In [14]:
%%px --target 0 --noblock
import dfca as DFCA
import json
import time
import torch
import numpy as np


with open("config_dfca_gi.json", "r") as read_file:
    config = json.load(read_file)

config['lr'] = 0.005
config['num_epochs'] = 150
res_final = []

for seed in seeds:
    print(f"---------------------------------DFCA-GI, lr:0.005 , seed: {seed}---------------------------------")
    start_time = time.time()
    config['data_seed'] = seed
    config['train_seed'] = config['data_seed']

    print("config:",config)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    exp = DFCA.TrainEMNISTCluster(config, device)
    exp.setup()
    res = exp.run()
    res_final.append([[r['test']['loss'] for r in res], [r['test']['acc'] for r in res], [r['test']['cl_acc'] for r in res]])
    del exp
    duration = (time.time() - start_time)
    print("---train cluster Ended in %0.2f hour (%.3f sec) " % (duration/float(3600), duration))

res_mean = np.mean(res_final, axis=0).tolist()
dfca_gi.append(res_mean)



<AsyncResult(%px): pending>

Running the ifca experiment

In [15]:
%%px --target 1 --noblock
import ifca as IFCA
import json
import time
import torch
import numpy as np

with open("config_ifca.json", "r") as read_file:
    config = json.load(read_file)

config['lr'] = 0.1
config['num_epochs'] = 150
res_final = []

for seed in seeds:
    print(f"---------------------------------IFCA, lr:0.1 , seed: {seed}---------------------------------")
    start_time = time.time()
    config['data_seed'] = seed
    config['train_seed'] = config['data_seed']

    print("config:",config)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    exp = IFCA.TrainEMNISTCluster(config, device)
    exp.setup()
    res = exp.run()
    res_final.append([[r['test']['loss'] for r in res], [r['test']['acc'] for r in res], [r['test']['cl_acc'] for r in res]])
    del exp
    duration = (time.time() - start_time)
    print("---train cluster Ended in %0.2f hour (%.3f sec) " % (duration/float(3600), duration))

res_mean = np.mean(res_final, axis=0).tolist()
ifca.append(res_mean)



<AsyncResult(%px): pending>

## Results

In [16]:
rc.wait_interactive()
_.display_outputs()
__.display_outputs()

dfca_gi = rc[0].pull('dfca_gi', block=True)
ifca = rc[1].pull('ifca', block=True)

unknown:   0%|          | 0/2 [00:00<?, ?tasks/s]

KeyboardInterrupt: 

In [ ]:

r_ifca = {}
r_dfca_gi = {}

r_ifca['test_loss'] = []
r_ifca['test_acc'] = []
r_ifca['cl_acc'] = []
for i in range(1):
    r_ifca['test_loss'].append(ifca[i][0])
    r_ifca['test_acc'].append(ifca[i][1])
    r_ifca['cl_acc'].append(ifca[i][2])

r_dfca_gi['test_loss'] = []
r_dfca_gi['test_acc'] = []
r_dfca_gi['cl_acc'] = []
for i in range(1):
    r_dfca_gi['test_loss'].append(dfca_gi[i][0])
    r_dfca_gi['test_acc'].append(dfca_gi[i][1])
    r_dfca_gi['cl_acc'].append(dfca_gi[i][2])

os.makedirs('graphs', exist_ok=True)
l=0
plt.figure(figsize=(10,5))
plt.plot(r_ifca['test_loss'][l], label='ifca', color='blue')
plt.plot(r_dfca_gi['test_loss'][l], label='dfca-gi', color='green')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title(f'Test Loss per Epoch')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join('graphs', '2_train_loss.png'))

plt.figure(figsize=(10,5))
plt.plot(r_ifca['test_acc'][l], label='ifca', color='blue')
plt.plot(r_dfca_gi['test_acc'][l], label='dfca-gi', color='green')
plt.xlabel('epoch')
plt.ylabel('test accuracy')
plt.title(f'Test Accuracy per Epoch')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join('graphs', '2_test_acc.png'))

plt.figure(figsize=(10,5))
plt.plot(r_ifca['cl_acc'][l][:31], label='ifca', color='blue')
plt.plot(r_dfca_gi['cl_acc'][l][:31], label='dfca-gi', color='green')
plt.xlabel('epoch')
plt.ylabel('cluster acc')
plt.title(f'Cluster Accuracy per Epoch')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join('graphs', '2_cluster_acc.png'))

IndexError: list index out of range